[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Khipucamayoc/Quipucamayoc/blob/master/colabs/GPT_2_amediavoz_training.ipynb)

In [ ]:
!nvidia-smi

Tue Mar 17 17:25:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
!pip install regex

In [ ]:
import codecs
import regex
import json

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"0ByELE8V-uYi2TkxlRmZERkY3eVpONHJXSzFEYnZaY3ctMmdr"}) 
lyrics_filename = '/content/amediavoz_poems.json'
downloaded.GetContentFile(lyrics_filename)

In [ ]:
# create raw string from JSON
raw_str_clean = ''
with open(lyrics_filename) as f:
    data = json.load(f)
    for item in data:
      for poem in item['poems']:
        for line in poem['poem']:
          raw_str_clean += line + '\n'

In [ ]:
# remove spaces or tabs from any empty lines
newlines_with_space_pattern = regex.compile(r'(\n[[:blank:]]+\n)')
raw_str_clean = newlines_with_space_pattern.sub(r'\n\n', raw_str_clean)

In [ ]:
# change three or more new lines to two
newlines_pattern = regex.compile(r'(\n[\n ]*\n)')

newlines_2 = newlines_pattern.sub(r'\n\n', raw_str_clean)
# print(newlines_2)

In [ ]:
file = codecs.open("text_clean.txt", "w", "utf-8")
file.write(newlines_2)
file.close()

In [ ]:
# Snippet based on https://colab.sandbox.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

# Download the small GPT-2 model.
# It's possible to finetune either the "124M" or "355M" model.
# The "774M" and "1558M" models can't currently been finetuned in Colab.
gpt2.download_gpt2(model_name="124M")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 651Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 108Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 982Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 78.4Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 444Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 213Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 234Mit/s]                                                       


In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset='text_clean.txt',
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/checkpoint/run1/ "/content/drive/My Drive/Paola/AMI Grants: Paola Torres Núñez Del Prado/amediavoz/"